In [1]:
import ipyfan
from ipyfan.utils import norm_fn, rgba2rgb
import numpy as np
import ipywidgets as widgets
import glob
from pathlib import Path
import os
import matplotlib.pyplot as plt
from ipyfan.iis_model import ritm_model as iis_model  # AI

ipyfan.ritm.isegm.model.is_hrnet_model
torch.nn.modules.batchnorm


In [2]:
data_dir = 'figs'  # it is not yet very robust to directories with images of different sizes
list_paths = sorted(glob.glob(data_dir + '/*.png'))

In [3]:
def load_img(path):
    return plt.imread(path)*255
    
out = ('.png', '_annotation.png')

validate_button = widgets.Button(
    description='Validate',
    disabled=False)
previous_button = widgets.Button(
    description='Previous',
    disabled=False)
progress = widgets.IntProgress(
    value=0,
    min=0,
    max=len(list_paths)-1,
    description='Progress:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'black'},
    orientation='horizontal'
)

def validate(b):
    global w
    in_path = Path(list_paths[progress.value])
    out_path = in_path.parent / 'annotations' / in_path.name.replace(out[0], out[1])
    out_path.parent.mkdir(exist_ok=True, parents=True)

    if np.sum(w.annI[..., :3]) > 0:  # only save if something was masked
        plt.imsave(str(out_path), w.annI)
    else:
        # Annotation is empty, delete the corresponding file if it exists
        if os.path.exists(out_path):
            os.remove(out_path)

    progress.value += 1

    global curr_im
    curr_im = load_img(list_paths[progress.value])
    w.reset()
    w.set_image(curr_im, None)

def previous(b):
    global w
    progress.value -= 1

    global curr_im
    curr_im = load_img(list_paths[progress.value])
    w.reset()
    w.set_image(curr_im, None)

validate_button.on_click(validate)
previous_button.on_click(previous)

In [4]:
s = ipyfan.FullSegmenter(iis_model, load_img(list_paths[progress.value]), None, 400)
w = s.w
widgets.VBox([s.get_widget(), widgets.HBox([validate_button, previous_button, progress])])